question 1: 

Task: use 17 industry portfolio as regressor, estimate HML and MOM 

1. create HML and MOM using 6 portfolios
2. the returns of HML is factor return of value portoflio; the returns of MOM is factor return of momentum portfolio

*portfolio of each year seems to be constructed based on data at the end of June of the same year, so do the same for this?

For each of HML and MOM: 
1. regress 17 industry portfolio on HML/MOM

2. run one of each type of regression: linear, ridge, and LASSO with all regressor. Pick best 2 out 3

3. best 2 x (forward + hybrid) + random forest 


Compare selection criteria across models and across (1a), (1b), and (1c)
1. For selection of optimal model with the same number of predictors we only use in-sample SSE
2. For overall selection we use CV SSE, AIC, and BIC (in slide)

  

select three best models for HML and for MOM separately for question 1 

  

question 2: 

Task: use 17 industry portfolio as regressor, Comment on the ease or difficulty of replicating the components as opposed to the entire return 

1. use three best model for HML and compare that with accuracy of SV+BV and SG+BG 

2. use three best model for MOM and compare that with accuracy of SH+BH and SL+BL 

  

question 3: 

Task: combine the three procedures, and test accuracy on HML and MOM vs each of the component 

if still have word count, can test accuracy on long-side and short side and/or do pairs combinations of three procedures 


In [56]:
import numpy as np
import pandas as pd
from itertools import combinations
import calendar
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

In [57]:
mom = pd.read_csv("factor_return_mom.csv")

hml = pd.read_csv("factor_return_value.csv")

#The 6 Portfolios Formed on Size and Momentum (contains SH BH SL BL)
size_momentum = pd.read_csv("6_Portfolios_size_mom.csv")
size_momentum = size_momentum.iloc[:1174]
size_momentum['Unnamed: 0'] = size_momentum['Unnamed: 0'].astype(int)

#The 6 Portfolios Formed on Size and Value (contains SV BV SG BG)
size_value = pd.read_csv("6_Portfolios_size_value.csv")
#small lo seems to be small growth

#The 17 Industry Portfolios (regressors)
seventeen = pd.read_csv("17_Industry_Portfolios_cleaned.csv")

# Display the first few rows of the DataFrame
print(hml.head())
print(mom.head())
print(size_momentum.head())
print(size_value.head())
print(seventeen.head())

   Unnamed: 0   HML
0      192607 -2.43
1      192608  3.82
2      192609  0.13
3      192610  0.70
4      192611 -0.51
   Unnamed: 0    MOM
0      192701   0.36
1      192702  -2.14
2      192703   3.61
3      192704   4.30
4      192705  30.00
   Unnamed: 0  SMALL LoPRIOR  ME1 PRIOR2  SMALL HiPRIOR  BIG LoPRIO  \
0      192701           0.08        3.57           0.40       -0.40   
1      192702           7.01        6.48           5.51        7.27   
2      192703          -3.46       -2.88          -2.06       -3.57   
3      192704          -0.75       -0.48           3.32       -2.72   
4      192705           2.48       11.15           6.83        5.52   

   ME2 PRIOR2  B HiPRIOR  
0        0.26       0.00  
1        3.80       4.47  
2       -0.18       2.26  
3        0.75       1.81  
4        4.96       7.18  
   Unnamed: 0  SMALL LoBM  ME1 BM2  SMALL HiBM  BIG LoBM  ME2 BM2  BIG HiBM
0      192607      1.0874   0.9081     -0.0695    5.7168   1.8971    2.0066
1      192608

In [58]:
#make everything start at 192701
hml = hml.iloc[6:]
size_value = size_value.iloc[6:]
seventeen = seventeen.iloc[6:]

print(hml.head())
print(mom.head())
print(size_momentum.head())
print(size_value.head())
print(seventeen.head())

# check if times are the same, or some dataset is missing certain dates

    Unnamed: 0   HML
6       192701  4.54
7       192702  2.94
8       192703 -2.61
9       192704  0.81
10      192705  4.73
   Unnamed: 0    MOM
0      192701   0.36
1      192702  -2.14
2      192703   3.61
3      192704   4.30
4      192705  30.00
   Unnamed: 0  SMALL LoPRIOR  ME1 PRIOR2  SMALL HiPRIOR  BIG LoPRIO  \
0      192701           0.08        3.57           0.40       -0.40   
1      192702           7.01        6.48           5.51        7.27   
2      192703          -3.46       -2.88          -2.06       -3.57   
3      192704          -0.75       -0.48           3.32       -2.72   
4      192705           2.48       11.15           6.83        5.52   

   ME2 PRIOR2  B HiPRIOR  
0        0.26       0.00  
1        3.80       4.47  
2       -0.18       2.26  
3        0.75       1.81  
4        4.96       7.18  
    Unnamed: 0  SMALL LoBM  ME1 BM2  SMALL HiBM  BIG LoBM  ME2 BM2  BIG HiBM
6       192701     -0.7991  -0.3782      3.9316   -0.2621   0.0321    4.0885
7    

In [59]:
size_momentum['Unnamed: 0'] = pd.to_datetime(size_momentum['Unnamed: 0'].astype(str), format='%Y%m').dt.to_period('M')
size_value['Unnamed: 0'] = pd.to_datetime(size_value['Unnamed: 0'].astype(str), format='%Y%m').dt.to_period('M')
hml['Unnamed: 0'] = pd.to_datetime(hml['Unnamed: 0'].astype(str), format='%Y%m').dt.to_period('M')
mom['Unnamed: 0'] = pd.to_datetime(mom['Unnamed: 0'].astype(str), format='%Y%m').dt.to_period('M')
seventeen['Unnamed: 0'] = pd.to_datetime(seventeen['Unnamed: 0'].astype(str), format='%Y%m').dt.to_period('M')

In [60]:
print(seventeen.columns)
seventeen.columns = seventeen.columns.str.strip() #strip spaces
predictors = seventeen.columns[1:]

Index(['Unnamed: 0', 'Food ', 'Mines', 'Oil  ', 'Clths', 'Durbl', 'Chems',
       'Cnsum', 'Cnstr', 'Steel', 'FabPr', 'Machn', 'Cars ', 'Trans', 'Utils',
       'Rtail', 'Finan', 'Other'],
      dtype='object')


In [61]:
# Set date as the index
seventeen.set_index('Unnamed: 0', inplace=True)
size_momentum.set_index('Unnamed: 0', inplace=True)
size_value.set_index('Unnamed: 0', inplace=True)
hml.set_index('Unnamed: 0', inplace=True)
mom.set_index('Unnamed: 0', inplace=True)

In [62]:
#group data into 5-year intervals
seventeen_grouped = seventeen.groupby(pd.Grouper(freq='5Y'))
size_momentum_grouped = size_momentum.groupby(pd.Grouper(freq='5Y'))
size_value_grouped = size_value.groupby(pd.Grouper(freq='5Y'))
mom_grouped = mom.groupby(pd.Grouper(freq='5Y'))
hml_grouped = hml.groupby(pd.Grouper(freq='5Y'))

/tmp/ipykernel_1936/903688633.py:2: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  seventeen_grouped = seventeen.groupby(pd.Grouper(freq='5Y'))
/tmp/ipykernel_1936/903688633.py:2: FutureWarning: Resampling with a PeriodIndex is deprecated. Cast index to DatetimeIndex before resampling instead.
  seventeen_grouped = seventeen.groupby(pd.Grouper(freq='5Y'))
/tmp/ipykernel_1936/903688633.py:3: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  size_momentum_grouped = size_momentum.groupby(pd.Grouper(freq='5Y'))
/tmp/ipykernel_1936/903688633.py:3: FutureWarning: Resampling with a PeriodIndex is deprecated. Cast index to DatetimeIndex before resampling instead.
  size_momentum_grouped = size_momentum.groupby(pd.Grouper(freq='5Y'))
/tmp/ipykernel_1936/903688633.py:4: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  size_value_grouped = si

In [63]:
group_labels = list(seventeen_grouped.groups.keys())

In [64]:
print(group_labels)

[Period('1927', '5Y-DEC'), Period('1932', '5Y-DEC'), Period('1937', '5Y-DEC'), Period('1942', '5Y-DEC'), Period('1947', '5Y-DEC'), Period('1952', '5Y-DEC'), Period('1957', '5Y-DEC'), Period('1962', '5Y-DEC'), Period('1967', '5Y-DEC'), Period('1972', '5Y-DEC'), Period('1977', '5Y-DEC'), Period('1982', '5Y-DEC'), Period('1987', '5Y-DEC'), Period('1992', '5Y-DEC'), Period('1997', '5Y-DEC'), Period('2002', '5Y-DEC'), Period('2007', '5Y-DEC'), Period('2012', '5Y-DEC'), Period('2017', '5Y-DEC'), Period('2022', '5Y-DEC')]


# Analysis (1a)

## 5-year data

### Linear regression

In [65]:
# linear (variable selection)

linear = LinearRegression(fit_intercept=False)

### forward stepwise selection

###### one variable

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

for pred in predictors:
    #choose one variable
    explanatory = seventeen_grouped[pred]
    predict_hml = pd.Series()
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values.reshape(-1, 1)

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1936/1787319043.py:24: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1936/1787319043.py:29: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1936/1787319043.py:24: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Trans
The MOM predictor with the lowest SSE is: Trans


/tmp/ipykernel_1936/1787319043.py:24: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1936/1787319043.py:29: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1936/1787319043.py:24: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [66]:
###### two variable

# first selected variable is Trans for both

#remove from predictors
predictors1 = predictors[~predictors.isin(['Trans'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

for pred in predictors1:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans']]
    predict_hml = pd.Series()
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1936/35664418.py:23: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1936/35664418.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


/tmp/ipykernel_1936/35664418.py:23: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1936/35664418.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1936/35664418.py:23: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.


The HML predictor with the lowest SSE is: Machn
The MOM predictor with the lowest SSE is: Cnstr


In [67]:
###### three variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)

#remove from predictors
predictorsh = predictors1[~predictors1.isin(['Machn'])]
predictorsm = predictors1[~predictors1.isin(['Cnstr'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1936/3751845197.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1936/3751845197.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1936/3751845197.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Steel
The MOM predictor with the lowest SSE is: Mines


/tmp/ipykernel_1936/3751845197.py:43: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1936/3751845197.py:43: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1936/3751845197.py:43: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [68]:
###### four variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)

#remove from predictors
predictorsh = predictorsh[~predictorsh.isin(['Steel'])]
predictorsm = predictorsm[~predictorsm.isin(['Mines'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1936/2373838400.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


/tmp/ipykernel_1936/2373838400.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1936/2373838400.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1936/2373838400.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Finan
The MOM predictor with the lowest SSE is: Finan


/tmp/ipykernel_1936/2373838400.py:44: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


In [69]:
###### five variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)
# four variable model is Trans + Machn + Steel + Finan (HML), Trans + Cnstr + Mines + Finan (MOM)

#remove from predictors
predictorsh = predictorsh[~predictorsh.isin(['Finan'])]
predictorsm = predictorsm[~predictorsm.isin(['Finan'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Finan']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1936/613824281.py:29: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1936/613824281.py:29: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1936/613824281.py:29: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operatio

The HML predictor with the lowest SSE is: Other
The MOM predictor with the lowest SSE is: Rtail


/tmp/ipykernel_1936/613824281.py:45: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1936/613824281.py:45: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


In [70]:
###### six variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)
# four variable model is Trans + Machn + Steel + Finan (HML), Trans + Cnstr + Mines + Finan (MOM)
# five variable model is Trans + Machn + Steel + Finan + Other (HML), \
# Trans + Cnstr + Mines + Finan + Rtail (MOM)

#remove from predictors
predictorsh = predictorsh[~predictorsh.isin(['Other'])]
predictorsm = predictorsm[~predictorsm.isin(['Rtail'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Finan', 'Rtail']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1936/2402805268.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1936/2402805268.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1936/2402805268.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Oil
The MOM predictor with the lowest SSE is: FabPr


In [71]:
###### seven variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)
# four variable model is Trans + Machn + Steel + Finan (HML), Trans + Cnstr + Mines + Finan (MOM)
# five variable model is Trans + Machn + Steel + Finan + Other (HML), \
# Trans + Cnstr + Mines + Finan + Rtail (MOM)
# six variable model is Trans + Machn + Steel + Finan + Other + Oil (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr (MOM)

#remove from predictors
predictorsh = predictorsh[~predictorsh.isin(['Oil'])]
predictorsm = predictorsm[~predictorsm.isin(['FabPr'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Oil']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1936/2660957166.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1936/2660957166.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1936/2660957166.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Chems
The MOM predictor with the lowest SSE is: Clths


/tmp/ipykernel_1936/2660957166.py:49: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1936/2660957166.py:49: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


In [72]:
###### eight variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)
# four variable model is Trans + Machn + Steel + Finan (HML), Trans + Cnstr + Mines + Finan (MOM)
# five variable model is Trans + Machn + Steel + Finan + Other (HML), \
# Trans + Cnstr + Mines + Finan + Rtail (MOM)
# six variable model is Trans + Machn + Steel + Finan + Other + Oil (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr (MOM)
# seven variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths (MOM)

#remove from predictors
predictorsh = predictorsh[~predictorsh.isin(['Chems'])]
predictorsm = predictorsm[~predictorsm.isin(['Clths'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1936/4154706649.py:35: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1936/4154706649.py:35: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1936/4154706649.py:35: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Rtail
The MOM predictor with the lowest SSE is: Chems


In [73]:
###### nine variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)
# four variable model is Trans + Machn + Steel + Finan (HML), Trans + Cnstr + Mines + Finan (MOM)
# five variable model is Trans + Machn + Steel + Finan + Other (HML), \
# Trans + Cnstr + Mines + Finan + Rtail (MOM)
# six variable model is Trans + Machn + Steel + Finan + Other + Oil (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr (MOM)
# seven variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths (MOM)
# eight variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems (MOM)

#remove from predictors
predictorsh = predictorsh[~predictorsh.isin(['Rtail'])]
predictorsm = predictorsm[~predictorsm.isin(['Chems'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1936/2104319118.py:37: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1936/2104319118.py:37: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1936/2104319118.py:37: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Utils
The MOM predictor with the lowest SSE is: Food


In [74]:
###### ten variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)
# four variable model is Trans + Machn + Steel + Finan (HML), Trans + Cnstr + Mines + Finan (MOM)
# five variable model is Trans + Machn + Steel + Finan + Other (HML), \
# Trans + Cnstr + Mines + Finan + Rtail (MOM)
# six variable model is Trans + Machn + Steel + Finan + Other + Oil (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr (MOM)
# seven variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths (MOM)
# eight variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems (MOM)
# nine variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food (MOM)

#remove from predictors
predictorsh = predictorsh[~predictorsh.isin(['Utils'])]
predictorsm = predictorsm[~predictorsm.isin(['Food'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 
                                     'Utils']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 
                                     'Food']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1936/2135745534.py:40: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


/tmp/ipykernel_1936/2135745534.py:40: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1936/2135745534.py:40: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1936/2135745534.py:40: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Cnsum
The MOM predictor with the lowest SSE is: Machn


/tmp/ipykernel_1936/2135745534.py:57: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1936/2135745534.py:57: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1936/2135745534.py:57: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [75]:
###### 11 variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)
# four variable model is Trans + Machn + Steel + Finan (HML), Trans + Cnstr + Mines + Finan (MOM)
# five variable model is Trans + Machn + Steel + Finan + Other (HML), \
# Trans + Cnstr + Mines + Finan + Rtail (MOM)
# six variable model is Trans + Machn + Steel + Finan + Other + Oil (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr (MOM)
# seven variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths (MOM)
# eight variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems (MOM)
# nine variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food (MOM)
# 10 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn (MOM)

#remove from predictors
predictorsh = predictorsh[~predictorsh.isin(['Cnsum'])]
predictorsm = predictorsm[~predictorsm.isin(['Machn'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 
                                     'Utils', 'Cnsum']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 
                                     'Food', 'Machn']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1936/1372259047.py:42: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


/tmp/ipykernel_1936/1372259047.py:42: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1936/1372259047.py:42: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1936/1372259047.py:42: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Clths
The MOM predictor with the lowest SSE is: Steel


In [76]:
###### 12 variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)
# four variable model is Trans + Machn + Steel + Finan (HML), Trans + Cnstr + Mines + Finan (MOM)
# five variable model is Trans + Machn + Steel + Finan + Other (HML), \
# Trans + Cnstr + Mines + Finan + Rtail (MOM)
# six variable model is Trans + Machn + Steel + Finan + Other + Oil (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr (MOM)
# seven variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths (MOM)
# eight variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems (MOM)
# nine variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food (MOM)
# 10 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn (MOM)
# 11 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel (MOM)

#remove from predictors
predictorsh = predictorsh[~predictorsh.isin(['Clths'])]
predictorsm = predictorsm[~predictorsm.isin(['Steel'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 
                                     'Utils', 'Cnsum', 'Clths']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 
                                     'Food', 'Machn', 'Steel']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1936/2863303116.py:45: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1936/2863303116.py:45: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1936/2863303116.py:45: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Cnstr
The MOM predictor with the lowest SSE is: Oil


In [77]:
###### 13 variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)
# four variable model is Trans + Machn + Steel + Finan (HML), Trans + Cnstr + Mines + Finan (MOM)
# five variable model is Trans + Machn + Steel + Finan + Other (HML), \
# Trans + Cnstr + Mines + Finan + Rtail (MOM)
# six variable model is Trans + Machn + Steel + Finan + Other + Oil (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr (MOM)
# seven variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths (MOM)
# eight variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems (MOM)
# nine variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food (MOM)
# 10 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn (MOM)
# 11 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel (MOM)
# 12 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel + Oil (MOM)

#remove from predictors
predictorsh = predictorsh[~predictorsh.isin(['Cnstr'])]
predictorsm = predictorsm[~predictorsm.isin(['Oil'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 
                                     'Utils', 'Cnsum', 'Clths', 'Cnstr']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 
                                     'Food', 'Machn', 'Steel', 'Oil']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1936/412403489.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1936/412403489.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1936/412403489.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operatio

The HML predictor with the lowest SSE is: Cars
The MOM predictor with the lowest SSE is: Cnsum


In [78]:
###### 14 variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)
# four variable model is Trans + Machn + Steel + Finan (HML), Trans + Cnstr + Mines + Finan (MOM)
# five variable model is Trans + Machn + Steel + Finan + Other (HML), \
# Trans + Cnstr + Mines + Finan + Rtail (MOM)
# six variable model is Trans + Machn + Steel + Finan + Other + Oil (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr (MOM)
# seven variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths (MOM)
# eight variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems (MOM)
# nine variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food (MOM)
# 10 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn (MOM)
# 11 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel (MOM)
# 12 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel + Oil (MOM)
# 13 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr + Cars (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel + Oil + Cnsum (MOM)

#remove from predictors
predictorsh = predictorsh[~predictorsh.isin(['Cars'])]
predictorsm = predictorsm[~predictorsm.isin(['Cnsum'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 
                                     'Utils', 'Cnsum', 'Clths', 'Cnstr', 'Cars']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 
                                     'Food', 'Machn', 'Steel', 'Oil', 'Cnsum']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1936/2396443549.py:51: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1936/2396443549.py:51: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1936/2396443549.py:51: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Durbl
The MOM predictor with the lowest SSE is: Utils


/tmp/ipykernel_1936/2396443549.py:51: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1936/2396443549.py:68: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1936/2396443549.py:68: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [79]:
###### 15 variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)
# four variable model is Trans + Machn + Steel + Finan (HML), Trans + Cnstr + Mines + Finan (MOM)
# five variable model is Trans + Machn + Steel + Finan + Other (HML), \
# Trans + Cnstr + Mines + Finan + Rtail (MOM)
# six variable model is Trans + Machn + Steel + Finan + Other + Oil (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr (MOM)
# seven variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths (MOM)
# eight variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems (MOM)
# nine variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food (MOM)
# 10 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn (MOM)
# 11 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel (MOM)
# 12 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel + Oil (MOM)
# 13 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr + Cars (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel + Oil + Cnsum (MOM)
# 14 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr + Cars + Durbl (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel + Oil + Cnsum + Utils (MOM)

#remove from predictors
predictorsh = predictorsh[~predictorsh.isin(['Durbl'])]
predictorsm = predictorsm[~predictorsm.isin(['Utils'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 
                                     'Utils', 'Cnsum', 'Clths', 'Cnstr', 'Cars', 'Durbl']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 
                                     'Food', 'Machn', 'Steel', 'Oil', 'Cnsum', 'Utils']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1936/14378800.py:54: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1936/14378800.py:54: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1936/14378800.py:54: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.


The HML predictor with the lowest SSE is: Food
The MOM predictor with the lowest SSE is: Cars


/tmp/ipykernel_1936/14378800.py:71: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1936/14378800.py:71: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


In [80]:
###### 16 variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)
# four variable model is Trans + Machn + Steel + Finan (HML), Trans + Cnstr + Mines + Finan (MOM)
# five variable model is Trans + Machn + Steel + Finan + Other (HML), \
# Trans + Cnstr + Mines + Finan + Rtail (MOM)
# six variable model is Trans + Machn + Steel + Finan + Other + Oil (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr (MOM)
# seven variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths (MOM)
# eight variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems (MOM)
# nine variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food (MOM)
# 10 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn (MOM)
# 11 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel (MOM)
# 12 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel + Oil (MOM)
# 13 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr + Cars (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel + Oil + Cnsum (MOM)
# 14 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr + Cars + Durbl (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel + Oil + Cnsum + Utils (MOM)
# 15 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr + Cars + Durbl + Food (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel + Oil + Cnsum + Utils + Cars (MOM)

#remove from predictors
predictorsh = predictorsh[~predictorsh.isin(['Food'])]
predictorsm = predictorsm[~predictorsm.isin(['Cars'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 
                                     'Utils', 'Cnsum', 'Clths', 'Cnstr', 'Cars', 'Durbl', 'Food']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 
                                     'Food', 'Machn', 'Steel', 'Oil', 'Cnsum', 'Utils', 'Cars']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1936/2679972848.py:57: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1936/2679972848.py:57: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1936/2679972848.py:74: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Mines
The MOM predictor with the lowest SSE is: Durbl


/tmp/ipykernel_1936/2679972848.py:74: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


In [81]:
# Best Models
best_model_hml = []
best_model_mom = []
# first selected variable is Trans for both
best_model_hml.append('Trans')
best_model_mom.append('Trans')
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
best_model_hml.append(['Trans','Machn'])
best_model_mom.append(['Trans','Cnstr'])
# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)
best_model_hml.append(['Trans','Machn', 'Steel'])
best_model_mom.append(['Trans','Cnstr', 'Mines'])
# four variable model is Trans + Machn + Steel + Finan (HML), Trans + Cnstr + Mines + Finan (MOM)
best_model_hml.append(['Trans','Machn', 'Steel', 'Finan'])
best_model_mom.append(['Trans','Cnstr', 'Mines', 'Finan'])
# five variable model is Trans + Machn + Steel + Finan + Other (HML), \
# Trans + Cnstr + Mines + Finan + Rtail (MOM)
best_model_hml.append(['Trans','Machn', 'Steel', 'Finan', 'Other'])
best_model_mom.append(['Trans','Cnstr', 'Mines', 'Finan', 'Rtail'])
# six variable model is Trans + Machn + Steel + Finan + Other + Oil (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr (MOM)
best_model_hml.append(['Trans','Machn', 'Steel', 'Finan', 'Other', 'Oil'])
best_model_mom.append(['Trans','Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr'])
# seven variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths (MOM)
best_model_hml.append(['Trans','Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems'])
best_model_mom.append(['Trans','Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths'])
# eight variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems (MOM)
best_model_hml.append(['Trans','Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail'])
best_model_mom.append(['Trans','Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems'])
# nine variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food (MOM)
best_model_hml.append(['Trans','Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 'Utils'])
best_model_mom.append(['Trans','Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 'Food'])
# 10 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn (MOM)
best_model_hml.append(['Trans','Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 'Utils', 'Cnsum'])
best_model_mom.append(['Trans','Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 'Food', 'Machn'])
# 11 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel (MOM)
best_model_hml.append(['Trans','Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 'Utils', 'Cnsum', 
                       'Clths'])
best_model_mom.append(['Trans','Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 'Food', 'Machn', 
                       'Steel'])
# 12 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel + Oil (MOM)
best_model_hml.append(['Trans','Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 'Utils', 'Cnsum', 
                       'Clths', 'Cnstr'])
best_model_mom.append(['Trans','Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 'Food', 'Machn', 
                       'Steel', 'Oil'])
# 13 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr + Cars (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel + Oil + Cnsum (MOM)
best_model_hml.append(['Trans','Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 'Utils', 'Cnsum', 
                       'Clths', 'Cnstr', 'Cars'])
best_model_mom.append(['Trans','Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 'Food', 'Machn', 
                       'Steel', 'Oil', 'Cnsum'])
# 14 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr + Cars + Durbl (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel + Oil + Cnsum + Utils (MOM)
best_model_hml.append(['Trans','Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 'Utils', 'Cnsum', 
                       'Clths', 'Cnstr', 'Cars', 'Durbl'])
best_model_mom.append(['Trans','Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 'Food', 'Machn', 
                       'Steel', 'Oil', 'Cnsum', 'Utils'])
# 15 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr + Cars + Durbl + Food (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + \
# Steel + Oil + Cnsum + Utils + Cars (MOM)
best_model_hml.append(['Trans','Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 'Utils', 'Cnsum', 
                       'Clths', 'Cnstr', 'Cars', 'Durbl', 'Food'])
best_model_mom.append(['Trans','Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 'Food', 'Machn', 
                       'Steel', 'Oil', 'Cnsum', 'Utils', 'Cars'])
# 16 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr + Cars + Durbl + Food + Mines (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + \
# Steel + Oil + Cnsum + Utils + Cars + Durbl (MOM)
best_model_hml.append(['Trans','Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 'Utils', 'Cnsum', 
                       'Clths', 'Cnstr', 'Cars', 'Durbl', 'Food', 'Mines'])
best_model_mom.append(['Trans','Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 'Food', 'Machn', 
                       'Steel', 'Oil', 'Cnsum', 'Utils', 'Cars', 'Durbl'])
#17
best_model_hml.append(predictors.tolist())
best_model_mom.append(predictors.tolist())

In [ ]:
#choose best model using forward stepwise selection and out of sample SSE, AIC, BIC

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in best_model_hml:
    #choose one variable
    explanatory = seventeen_grouped[pred]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels)-1):
        if isinstance(pred, list):
            train = explanatory.get_group(group_labels[i]).values
            test = explanatory.get_group(group_labels[i+1]).values
        else:
            train = explanatory.get_group(group_labels[i]).values.reshape(-1, 1)
            test = explanatory.get_group(group_labels[i+1]).values.reshape(-1, 1)

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        #predict next 5 year HML
        predict = pd.Series(linear.predict(test).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM
for pred in best_model_mom:
    #choose one variable
    explanatory = seventeen_grouped[pred]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)-1):
        if isinstance(pred, list):
             train = explanatory.get_group(group_labels[i]).values
             test = explanatory.get_group(group_labels[i+1]).values
        else:
            train = explanatory.get_group(group_labels[i]).values.reshape(-1, 1)
            test = explanatory.get_group(group_labels[i+1]).values.reshape(-1, 1)

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        #predict next 5 year MOM
        predict = pd.Series(linear.predict(test).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

hml_to_exclude = hml_grouped.get_group(group_labels[0])
mom_to_exclude = mom_grouped.get_group(group_labels[0])
#actual response
hml_5yr_actual = hml[~hml.index.isin(hml_to_exclude.index)]
hml_5yr_actual = hml_5yr_actual.values
mom_5yr_actual = mom[~mom.index.isin(mom_to_exclude.index)]
mom_5yr_actual = mom_5yr_actual.values

#compare using out-of-sample SSE
sse_hml = ((predict_hml_df - hml_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)
sse_mom = ((predict_mom_df - mom_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)

# Number of observations in the actual data
n_hml = len(hml_5yr_actual)
n_mom = len(mom_5yr_actual)

k_values = list(range(1, 18))

# Calculate AIC and BIC for HML
aic_hml = [np.log(sse / n_hml) + 2 * k / n_hml for sse, k in zip(sse_hml, k_values)]
bic_hml = [np.log(sse / n_hml) + k * np.log(n_hml) / n_hml for sse, k in zip(sse_hml, k_values)]

# Calculate AIC and BIC for MOM
aic_mom = [np.log(sse / n_mom) + 2 * k / n_mom for sse, k in zip(sse_mom, k_values)]
bic_mom = [np.log(sse / n_mom) + k * np.log(n_mom) / n_mom for sse, k in zip(sse_mom, k_values)]

In [91]:
# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = sse_mom.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

# Choose optimal model (minimum AIC)
min_aic = aic_hml.index(min(aic_hml))
best_hml = best_model_hml[min_aic]
print(f"The HML predictor with the lowest AIC is: {best_hml}")
min_aic = aic_mom.index(min(aic_mom))
best_mom = best_model_mom[min_aic]
print(f"The MOM predictor with the lowest AIC is: {best_mom}")

# Choose optimal model (minimum BIC)
min_bic = bic_hml.index(min(bic_hml))
best_hml = best_model_hml[min_bic]
print(f"The HML predictor with the lowest BIC is: {best_hml}")
min_bic = bic_mom.index(min(bic_mom))
best_mom = best_model_mom[min_bic]
print(f"The MOM predictor with the lowest BIC is: {best_mom}")

The HML predictor with the lowest SSE is: ['Trans', 'Machn', 'Steel', 'Finan', 'Other']
The MOM predictor with the lowest SSE is: ['Trans', 'Cnstr']
The HML predictor with the lowest AIC is: ['Trans', 'Machn', 'Steel', 'Finan', 'Other']
The MOM predictor with the lowest AIC is: ['Trans', 'Cnstr']
The HML predictor with the lowest BIC is: ['Trans', 'Machn', 'Steel', 'Finan', 'Other']
The MOM predictor with the lowest BIC is: ['Trans', 'Cnstr']


In [ ]:
### backward stepwise selection

### hybrid stepwise selection

### Ridge

In [ ]:
# ridge and lasso (tune alpha)

### ridge

alpha_values = np.logspace(-6, 4, 13)

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

for a in alpha_values:

    predict_hml = pd.Series()
    predict_mom = pd.Series()

    for i in range(0,len(group_labels)-1):
        train = seventeen_grouped.get_group(group_labels[i]).values
        test = seventeen_grouped.get_group(group_labels[i+1]).values
        
        #estimate HML using 5 year data
        ridge_model = Ridge(alpha=a)
        ridge_model.fit(train, hml_grouped.get_group(group_labels[i]).values)
        
        #predict next 5 year HML
        predict = pd.Series(ridge_model.predict(test).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{a}'] = predict_hml
    
        #estimate MOM using 5 year data
        ridge_model = Ridge(alpha=a)
        ridge_model.fit(train, mom_grouped.get_group(group_labels[i]).values)
    
        #predict next 5 year MOM
        predict = pd.Series(ridge_model.predict(test).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{a}'] = predict_mom

#compare using out-of-sample SSE
sse_hml = ((predict_hml_df - hml_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)
sse_mom = ((predict_mom_df - mom_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)

# Number of observations in the actual data
n_hml = len(hml_5yr_actual)
n_mom = len(mom_5yr_actual)

k_values = list(range(1, 18))

# Calculate AIC and BIC for HML
aic_hml = [np.log(sse / n_hml) + 2 * k / n_hml for sse, k in zip(sse_hml, k_values)]
bic_hml = [np.log(sse / n_hml) + k * np.log(n_hml) / n_hml for sse, k in zip(sse_hml, k_values)]

# Calculate AIC and BIC for MOM
aic_mom = [np.log(sse / n_mom) + 2 * k / n_mom for sse, k in zip(sse_mom, k_values)]
bic_mom = [np.log(sse / n_mom) + k * np.log(n_mom) / n_mom for sse, k in zip(sse_mom, k_values)]

#calculate CV SSE, AIC, BIC

### lasso

In [ ]:
#random forest